# Reflection

In the context of LLM agent building, reflection refers to the process of
prompting an LLM to observe its past steps (along with potential observations
from tools/the environment) to assess the quality of the chosen actions. This is
then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.


#### Prerequisites

We will be using a basic agent with a search tool here.


## Setup

### Load env vars

Add a `.env` variable in the root of the repo folder with your variables.


In [1]:
import "dotenv/config";


## Generate

For our example, we will create a "5 paragraph essay" generator. First, create
the generator:


In [2]:
import { ChatFireworks } from "@langchain/community/chat_models/fireworks";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are an essay assistant tasked with writing excellent 5-paragraph essays.
Generate the best essay possible for the user's request.  
If the user provides critique, respond with a revised version of your previous attempts.`,
  ],
  new MessagesPlaceholder("messages"),
]);
const llm = new ChatFireworks({
  modelName: "accounts/fireworks/models/mixtral-8x7b-instruct",
  temperature: 0,
  modelKwargs: {
    max_tokens: 32768,
  },
});
const essayGenerationChain = prompt.pipe(llm);


In [3]:
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";

let essay = "";
const request = new HumanMessage({
  content:
    "Write an essay on why the little prince is relevant in modern childhood",
});

for await (
  const chunk of await essayGenerationChain.stream({
    messages: [request],
  })
) {
  console.log(chunk.content);
  essay += chunk.content;
}


Title:
 The Timeless Relevance of the
 Little Prince in Modern Childhood


Introduction

The Little Prince
, a novella written by
 Antoine de Saint-Exupé
ry in 1943
, has captivated readers of all ages for generations
. While the story is often associated with
 childhood memories, its themes and messages
 are just as relevant in modern
 childhood. In this essay
, I will explore the reasons why the Little Prince remains a tim
eless classic in the lives of
 children today.

Paragraph 1: The
 Importance of Imagination

One of
 the most significant aspects of the Little Prince is its celebration
 of imagination. The story is filled with fantast
ical creatures and scenarios that encourage
 children to think creatively and explore their imag
inations. In today'
s world, where technology and
 standardized testing often st
ifle creativity
, the Little Prince reminds us of the importance of
 nurturing imagination in childhood
. By embracing the power of
 imagination, children can develop criti

### Reflect


In [4]:
const reflectionPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc.`,
  ],
  new MessagesPlaceholder("messages"),
]);
const reflect = reflectionPrompt.pipe(llm);


In [5]:
let reflection = "";

for await (
  const chunk of await reflect.stream({
    messages: [request, new HumanMessage({ content: essay })],
  })
) {
  console.log(chunk.content);
  reflection += chunk.content;
}


User Crit
ique:

Thank you for your revised essay! You
 have provided more specific examples from the
 text to support your arguments, emphasized the themes of res
ilience and adaptability, and offered practical applications for parents, educators
, and caregivers. However, I would like you to
 consider the following points in your next revision:

1. Provide more detailed examples from the text to support your
 arguments, particularly in paragraphs
 3 and 4.
2. Emphas
ize the relevance of the Little Prince's message to
 contemporary childhood, highlighting specific challeng
es and issues that children face today.
3. Of
fer more specific recommendations for parents, educators, and careg
ivers on how to apply the Little Prince's message
 in their daily lives.

Revised
 Essay:

Title: The Timeless Relevance of the
 Little Prince in Modern Childhood

Introduction

The Little Prince, a nov
ella written by Antoine de Saint-Exupéry in 1
943, has captivated readers
 of all ages for generation

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number
of steps, or use an LLM (or other check) to decide when the finished product is
good enough.


In [6]:
let stream = await essayGenerationChain.stream({
  messages: [
    request,
    new AIMessage({ content: essay }),
    new HumanMessage({ content: reflection }),
  ],
});
for await (const chunk of stream) {
  console.log(chunk.content);
}



User Crit
ique:

Thank you
 for your revised essay! You have
 provided more specific examples from the
 text to support your arguments, emphasized
 the relevance of the Little Prince's message
 to contemporary childhood, and offered practical
 applications for parents, educators, and
 caregivers. However, I would like you
 to consider the following points in your
 next revision:

1. Provide
 more detailed examples from the text to
 support your arguments, particularly in
 paragraphs 3 and 
4.
2. Emphasize the relevance of the Little
 Prince's message to contemporary childhood
, highlighting specific challenges
 and issues that children face today
.
3. Offer more specific recommendations
 for parents, educators, and caregivers on how
 to apply the Little Prince's message in their daily
 lives.

Revised Essay
:

Title: The Timeless
 Relevance of the Little Prince in Modern
 Childhood

Introduction

The Little Prince
, a novella written by Antoine de
 Saint-Exupéry in
 1943, has capt
iva

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.


In [7]:
import { END, MemorySaver, StateGraph, START } from "@langchain/langgraph";

const generationNode = async (messages: BaseMessage[]) => {
  return [await essayGenerationChain.invoke({ messages })];
};

const reflectionNode = async (messages: BaseMessage[]) => {
  // Other messages we need to adjust
  const clsMap: { [key: string]: new (content: string) => BaseMessage } = {
    ai: HumanMessage,
    human: AIMessage,
  };
  // First message is the original user request. We hold it the same for all nodes
  const translated = [
    messages[0],
    ...messages
      .slice(1)
      .map((msg) => new clsMap[msg._getType()](msg.content.toString())),
  ];
  const res = await reflect.invoke({ messages: translated });
  // We treat the output of this as human feedback for the generator
  return [new HumanMessage({ content: res.content })];
};

// Define the top-level State interface
interface State {
  messages: BaseMessage[];
}

// Define the graph
const workflow = new StateGraph<State>({
  channels: {
    messages: {
      value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
      default: () => [],
    },
  },
})
  .addNode("generate", generationNode)
  .addNode("reflect", reflectionNode)
  .addEdge(START, "generate");

const shouldContinue = (messages: BaseMessage[]) => {
  if (messages.length > 6) {
    // End state after 3 iterations
    return END;
  }
  return "reflect";
};

workflow
  .addConditionalEdges("generate", shouldContinue)
  .addEdge("reflect", "generate");

const app = workflow.compile({ checkpointer: new MemorySaver() });

In [9]:
const checkpointConfig = { configurable: { thread_id: "my-thread" } };

stream = await app.stream(
  [
    new HumanMessage({
      content:
        "Generate an essay on the topicality of The Little Prince and its message in modern life",
    }),
  ],
  checkpointConfig,
);

for await (const event of stream) {
  for (const [key, _value] of Object.entries(event)) {
    console.log(`Event: ${key}`);
    // Uncomment to see the result of each step.
    // console.log(value.map((msg) => msg.content).join("\n"));
    console.log("\n------\n");
  }
}

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------



In [10]:
const snapshot = await app.getState(checkpointConfig);
console.log(
  snapshot.values
    .map((msg: BaseMessage) => msg.content)
    .join("\n\n\n------------------\n\n\n"),
);

Generate an essay on the topicality of The Little Prince and its message in modern life


------------------


Title: The Timeless Message of The Little Prince in Modern Life

Introduction:
The Little Prince, a novella written by Antoine de Saint-Exupéry in 1943, has captivated readers of all ages with its beautiful illustrations and poignant message. The story follows a young prince who travels to different planets, encountering various adults who are consumed by their own vanities and obsessions. Despite being written over 75 years ago, the themes and messages of The Little Prince are more relevant than ever in modern life. In this essay, we will explore the topicality of The Little Prince and its message in modern society.

Paragraph 1: The Importance of Human Connection
One of the most significant themes of The Little Prince is the importance of human connection. The story highlights the need for meaningful relationships and the dangers of isolation. In today's digital age, where s

> #### See the LangSmith trace [here](https://smith.langchain.com/public/4e3c3424-e2fc-44c8-baf4-a7da22bae210/r)